In [1]:
import pandas as pd
import sklearn
import re
import time, timeit
import numpy as np
import matplotlib as plt
import scipy

In [2]:
fname = r"C:\Users\hyh6hhy\Desktop\nlp-4\sqlResult_1558435.csv"

In [3]:
data = pd.read_csv(fname, encoding='gb18030')

In [4]:
data.head()

,id,author,source,content,feature,title,url
0,89617,NaN,快科技@http://www.kkj.cn/,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""37""...",小米MIUI 9首批机型曝光：共计15款,http://www.cnbeta.com/articles/tech/623597.htm
1,89616,NaN,快科技@http://www.kkj.cn/,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""15""...",骁龙835在Windows 10上的性能表现有望改善,http://www.cnbeta.com/articles/tech/623599.htm
2,89615,NaN,快科技@http://www.kkj.cn/,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""18""...",一加手机5细节曝光：3300mAh、充半小时用1天,http://www.cnbeta.com/articles/tech/623601.htm
3,89614,NaN,新华社,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n,"{""type"":""国际新闻"",""site"":""环球"",""commentNum"":""0"",""j...",葡森林火灾造成至少62人死亡 政府宣布进入紧急状态（组图）,http://world.huanqiu.com/hot/2017-06/10866126....
4,89613,胡淑丽_MN7479,深圳大件事,（原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\r\n@深圳交警微博称：昨日清...,"{""type"":""新闻"",""site"":""网易热门"",""commentNum"":""978"",...",44岁女子约网友被拒暴雨中裸奔 交警为其披衣相随,http://news.163.com/17/0618/00/CN617P3Q0001875...


In [5]:
data.loc[0,['feature']]

feature    {"type":"科技","site":"cnbeta","commentNum":"37"...
Name: 0, dtype: object

# add clock

In [6]:
def clock(func):
    def clocked(*args, **kwargs):
        t0 = timeit.default_timer()
        result = func(*args, **kwargs)
        elapsed = timeit.default_timer() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print(elapsed,'s')
        #print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked

# INPUT: Which is more useful, more efficient ?

In [7]:
@clock
def get_size(data, attributes):
    size = 0
    data = data[attributes]
    for i in range(len(data)):
        if isinstance(data[i],str):
            size = size + len(data[i])
    return size, size/len(data)

attributes = ['content','feature', 'title','author','url']
attr_choosen = ['content']
#attr_choosen = ['content','feature', 'title','author','url']
attr_choosen = ['content']
show = [print( i, get_size(data,i)) for i in attributes]

4.277370116 s
content (38729226, 432.1927665130397)
2.687388330000001 s
feature (53450994, 596.4780439901351)
3.8331849430000027 s
title (2123231, 23.69386570845097)
2.293857959999997 s
author (505494, 5.6409815759225985)
2.360930752999998 s
url (7472901, 83.39267500641662)


# 不平衡数据处理：重复但有过拟合风险；剪掉浪费；可以重组。imbalanced 用作baseline。TF-IDF不能像word2vec展示词之间的关系

In [8]:
xinhua_news = data[data["source"] == "新华社"]
len(xinhua_news)/len(data)

0.8778051801676133

In [9]:
data["source"] = data["source"].fillna("")
data_pos = data[data["source"].str.contains("新华社")]
data_neg = data[data["source"].str.contains("新华社") == 0]
NUM = 6000
data_pos = data_pos.sample(n = (int)(NUM/2))
data_neg = data_neg.sample(n = (int)(NUM/2))

data_sampled = data_pos.append(data_neg)

data_sampled.head()

,id,author,source,content,feature,title,url
56707,32905,殷博古,新华社,新华社照片，华盛顿，2017年5月10日\n（国际）（3）美国和格鲁吉亚签署情报保护协定\n...,"{""type"":""政治法律"",""site"":""新华社"",""url"":""http://home...",（国际）（3）美国和格鲁吉亚签署情报保护协定,http://home.xinhua-news.com/gdsdetailxhsnew/20...
39843,49769,刘秀玲,新华社,【新华社微特稿】《西日本新闻》２４日援引内部人士的话报道，旨在为“慰安妇”受害者提供所谓...,"{""type"":""政治法律"",""site"":""新华社"",""url"":""http://home...",【微特稿·时事与军事】日韩“慰安妇”基金面临夭折？,http://home.xinhua-news.com/gdsdetailxhsnew/19...
76148,13464,毛伟豪,新华社,新华社北京５月２６日电（记者毛伟豪）从北京到保定，１５０公里的距离，４２分钟的高铁车程。...,"{""type"":""社会"",""site"":""新华社"",""url"":""http://home.x...",（京津冀在行动）１３８名北京儿科专家为何坐高铁跨城上班？,http://home.xinhua-news.com/gdsdetailxhsnew/21...
44751,44861,张莹,新华社,新华社布鲁塞尔４月２８日新媒体专电欧盟委员会２７日发布“欧洲城市水资源管理地图集”，详细展示...,"{""type"":""科技"",""site"":""新华社"",""url"":""http://home.x...",（新华简讯）欧盟发布“欧洲城市水资源管理地图集”,http://home.xinhua-news.com/gdsdetailxhsnew/19...
18437,71175,J. Luis Cuesta / Cordon P/SIPA,新华社,新华社照片，外代，2017年4月6日\n（外代二线）马德里动物园的大熊猫“竹莉娜”首次与观众...,"{""type"":""其它"",""site"":""新华社"",""url"":""http://home.x...",[12]（外代二线）马德里动物园的大熊猫“竹莉娜”首次与观众正式见面,http://home.xinhua-news.com/gdsdetailxhsnew/18...


# 去标点，加label

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
import jieba
import re

In [11]:

def cut(string):
    return " ".join(jieba.cut(re.sub(r'[^\w\s]', " ", string.strip())))   #string
    #return ' '.join(jieba.cut(re.sub(r'[^\w\s]',' ',string.strip())))
def preprocessing(data):
    for attri in attributes:
        data[attri] = data[attri].fillna('').apply(cut)  #fillna('').
    data['is_xinhua'] = np.where(data['source'].str.contains('新华'), 1, 0) #.str.contains('新华')
    x_inputs = data.loc[: ,attr_choosen]
    y_inputs = data['is_xinhua']
    return x_inputs, y_inputs
x_inputs, y_inputs = preprocessing(data_sampled)


Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\hyh6hhy\AppData\Local\Temp\jieba.cache
Loading model cost 1.409 seconds.
Prefix dict has been built succesfully.


In [12]:
type(x_inputs['content'].values)

numpy.ndarray

# Text Representation with tf-idfvectorizer

In [13]:
@clock
def vectorize(x_inputs):
    vectorizer = TfidfVectorizer(max_features=5000, token_pattern=r"(?u)\b\w+\b", max_df = 1.0)# (?u)进入re.u，unicode匹配模式，https://stackoverflow.com/questions/35043085/what-does-u-do-in-a-regex
    #vectorizer = TfidfVectorizer(token_pattern = "(?u)\b\w+\b",max_features=5000, max_df = 1.0)
    matrix = None
    for attri in attr_choosen:
        #print(x_inputs[attri].values)
        #X = vectorizer.fit_transform(x_inputs[attri].values)
        vec_fea = vectorizer.fit_transform(x_inputs[attri].values)
        #print("vocabulary\n",vectorizer.vocabulary_, len(vectorizer.vocabulary_))
        #print("stop_words\n",vectorizer.stop_words_, len(vectorizer.stop_words_))
        
        if matrix == None:
            matrix = vec_fea
        else:
            matrix = np.hstack((matrix, vec_fea))
    return matrix

x_vecs = vectorize(x_inputs)

3.1364556289999967 s


In [14]:
x_vecs.shape

(6000, 5000)

In [15]:
y_inputs

56707    1
39843    1
76148    1
44751    1
18437    1
        ..
9870     0
2544     0
902      0
7584     0
4523     1
Name: is_xinhua, Length: 6000, dtype: int32

In [16]:
from sklearn.model_selection import train_test_split
x_train,x_test, y_train, y_test = train_test_split(
    x_vecs , y_inputs, train_size = 0.8, test_size=0.2)

# Evaluation

In [17]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

In [18]:
def get_performance(model, x, y):
    y_pred = model.predict(x)
    print("f1 score is {}".format(f1_score(y, y_pred)))
    print("accuracy score is {}".format(accuracy_score(y,y_pred)))
    print("precision score is {}".format(precision_score(y, y_pred)))
    print("recall score is {}".format(recall_score(y, y_pred)))
    print("roc auc score is {}".format(roc_auc_score(y, y_pred)))
    print("confusion matrix :\n {}".format(confusion_matrix(y, y_pred)))

In [ ]:
#import model

In [74]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier

# KNN*

In [20]:
@clock
def KNN(x_train, y_train):
    neigh = KNeighborsClassifier(n_neighbors=3)
    model = neigh.fit(x_train, y_train)
    return model
get_performance(KNN(x_train, y_train),x_test, y_test )

0.06580940599999963 s
f1 score is 0.6533333333333333
accuracy score is 0.74
precision score is 0.9966101694915255
recall score is 0.4859504132231405
roc auc score is 0.7421348704771165
confusion matrix :
 [[594   1]
 [311 294]]


In [36]:
parameters = {'n_neighbors':list(range(1,6))}
@clock
def KNN(x_train, y_train):
    clf = KNeighborsClassifier()
    grid = GridSearchCV(clf, parameters, cv=2, scoring = 'accuracy', n_jobs = -1)
    clf = grid.fit(x_train, y_train)
    #clf.fit(x_train, y_train)
    print(clf.best_params_)
    return clf
get_performance(KNN(x_train, y_train),x_test, y_test )

{'n_neighbors': 1}
8.043894962999957 s
f1 score is 0.7655786350148367
accuracy score is 0.8025
precision score is 0.9532019704433498
recall score is 0.6396694214876033
roc auc score is 0.8038683241891798
confusion matrix :
 [[576  19]
 [218 387]]


# Linear Regression

In [43]:
@clock
def LReg(x_train, y_train):
    reg = LinearRegression().fit(x_train, y_train)
    
    return reg
#get_performance(LReg(x_train, y_train), x_test,y_test)

# Logistic Regression

In [35]:
@clock
def logreg(x_train, y_train):
    clf = LogisticRegression(random_state=0, solver='liblinear').fit(x_train, y_train)
    return clf
get_performance(logreg(x_train, y_train), x_test,y_test)

0.10947630900000149 s
f1 score is 0.976311336717428
accuracy score is 0.9766666666666667
precision score is 1.0
recall score is 0.9537190082644628
roc auc score is 0.9768595041322314
confusion matrix :
 [[595   0]
 [ 28 577]]


In [42]:
parameters = {'solver':('liblinear', 'sag','saga')}
@clock
def logreg(x_train, y_train):
    clf = LogisticRegression(random_state=0)
    grid = GridSearchCV(clf, parameters, cv=2, scoring = 'accuracy', n_jobs = -1)
    clf = grid.fit(x_train, y_train)
    #clf.fit(x_train, y_train)
    print(clf.best_params_)
    return clf
get_performance(logreg(x_train, y_train), x_test,y_test)

{'solver': 'liblinear'}
0.4369922470000347 s
f1 score is 0.976311336717428
accuracy score is 0.9766666666666667
precision score is 1.0
recall score is 0.9537190082644628
roc auc score is 0.9768595041322314
confusion matrix :
 [[595   0]
 [ 28 577]]


# SVM

In [48]:
@clock
def svm(x_train, y_train):
    clf = SVC(random_state=0).fit(x_train, y_train)
    return clf
get_performance(svm(x_train, y_train), x_test,y_test)

C:\Users\hyh6hhy\Miniconda3\envs\stanfordnlp\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


35.38155877600002 s
f1 score is 0.6703601108033241
accuracy score is 0.5041666666666667
precision score is 0.5041666666666667
recall score is 1.0
roc auc score is 0.5
confusion matrix :
 [[  0 595]
 [  0 605]]


In [50]:
parameters = {'kernel':('linear', 'poly', 'rbf', 'sigmoid')}

@clock
def svm(x_train, y_train):
    clf = SVC(random_state=0)
    grid = GridSearchCV(clf, parameters, cv=2, scoring = 'accuracy', n_jobs = -1)
    clf = grid.fit(x_train, y_train)
    print(clf.best_params_)
    return clf
get_performance(svm(x_train, y_train), x_test,y_test)

{'kernel': 'linear'}
48.48124402500025 s
f1 score is 0.9848993288590604
accuracy score is 0.985
precision score is 1.0
recall score is 0.9702479338842975
roc auc score is 0.9851239669421488
confusion matrix :
 [[595   0]
 [ 18 587]]


# Naive Bayes*

In [59]:
@clock
def gnb(x_train, y_train):
    gnb = GaussianNB().fit(x_train.toarray(), y_train) #x needed to be dense
    return gnb
get_performance(gnb(x_train, y_train), x_test.toarray(),y_test)

1.0314007970000603 s
f1 score is 0.8575624082232012
accuracy score is 0.8383333333333334
precision score is 0.7714663143989432
recall score is 0.9652892561983472
roc auc score is 0.8372664768386694
confusion matrix :
 [[422 173]
 [ 21 584]]


In [67]:
parameters = {'var_smoothing':[1e-08, 1e-09, 1e-10, 1e-09-11]}

@clock
def gnb(x_train, y_train):
    gnb = GaussianNB()
    grid = GridSearchCV(gnb, parameters, cv=2, scoring = 'accuracy', n_jobs = -1)
    gnb = grid.fit(x_train.toarray(), y_train) #x needed to be dense
    print(gnb.best_params_)
    return gnb
get_performance(gnb(x_train, y_train), x_test.toarray(),y_test)

{'var_smoothing': 1e-09}
7.489706000000297 s
f1 score is 0.8575624082232012
accuracy score is 0.8383333333333334
precision score is 0.7714663143989432
recall score is 0.9652892561983472
roc auc score is 0.8372664768386694
confusion matrix :
 [[422 173]
 [ 21 584]]


# decision tree

In [73]:
@clock
def dec_tree(x_train, y_train):
    clf = tree.DecisionTreeClassifier().fit(x_train, y_train) 
    return clf
get_performance(dec_tree(x_train, y_train), x_test, y_test)


1.5752719710003475 s
f1 score is 0.977850697292863
accuracy score is 0.9775
precision score is 0.9706840390879479
recall score is 0.9851239669421488
roc auc score is 0.9774359330509063
confusion matrix :
 [[577  18]
 [  9 596]]


# Random Forest

In [75]:
@clock
def RF(x_train, y_train):
    clf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0).fit(x_train, y_train) 
    return clf
get_performance(RF(x_train, y_train), x_test, y_test)


0.35335307400009697 s
f1 score is 0.9107142857142858
accuracy score is 0.9166666666666666
precision score is 0.9902912621359223
recall score is 0.8429752066115702
roc auc score is 0.9172859226335163
confusion matrix :
 [[590   5]
 [ 95 510]]


In [78]:
parameters = {'n_estimators':[50, 100,200],'max_depth':[5,8,11,15,18]}

@clock
def RF(x_train, y_train):
    clf = RandomForestClassifier(random_state=0)
    grid = GridSearchCV(clf, parameters, cv=2, scoring = 'accuracy', n_jobs = -1)
    clf = grid.fit(x_train.toarray(), y_train) 
    print(clf.best_params_)
    return clf
get_performance(RF(x_train, y_train), x_test.toarray(),y_test)

{'max_depth': 18, 'n_estimators': 50}
63.33335296699988 s
f1 score is 0.9814814814814815
accuracy score is 0.9816666666666667
precision score is 1.0
recall score is 0.9636363636363636
roc auc score is 0.9818181818181818
confusion matrix :
 [[595   0]
 [ 22 583]]
